# Projeto de ***grafos***

Grupo 5:

*   Allane Oliveira
*   Karolayne Melo
*   Pedro Bezerra
*   Samuel Silva
*   Vitor Alencar




## Pacotes e Bibliotecas

In [ ]:
!pip install openpyxl pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import math
import random
import time
import os

## Grafos utilizados e funções auxiliares

### Grafos

Para conseguir representar as matrizes de adjacência dos grafos, foi utilizado o recurso DataFrames da Biblioteca Pandas.

In [ ]:
matriz_problema_km = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Km")
matriz_problema_min = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Min")
cidades = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Cidades")

### Funções Auxiliares

#### Função Calcular custo


A função `calcular_custo` serve para somar os pesos das arestas percorridas por um caminho no grafo, funcionando como a medida de qualidade das soluções no algoritmo genético ou memético.

Além disso, ela recebe uma lista de vértices que representa o percurso e percorre essa lista par a par, pegando o custo de cada aresta diretamente na matriz de adjacência do grafo. A cada passo, o custo acumulado é atualizado até que todo o caminho seja percorrido, retornando ao final o custo total.
##### Exemplo:
Em um grafo onde a aresta entre 1 -> 3 custa 2 e 3 -> 2 custa 3, a solução `[1,2,3]` vai gerar um custo final de valor 5.


In [ ]:
# Função para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        custo += grafo[v1][v2 - 1]
    return custo

#### Busca Local Melhoria 1









A função `busca_local_primeira_melhoria` realiza uma busca local utilizando a estratégia de primeira melhoria. Ela recebe uma solução inicial e gera vizinhos usando a heurística escolhida (como shift, swap ou inversão), avaliando cada vizinho assim que é produzido. Sempre que encontra um vizinho com custo menor que o da solução atual, ela imediatamente adota esse vizinho como nova solução, sem analisar os demais.

O processo se repete enquanto houver melhoria. Quando nenhum vizinho apresenta um custo melhor, a busca é encerrada. Ao final, a função retorna a melhor solução encontrada e seu custo correspondente.

##### Exemplo:
Se um vizinho reduzir o custo de 40 para 28, ele é aceito na hora, e a busca continua a partir dele.

In [ ]:
# Executa busca local usando a estratégia de primeira melhoria.
# 'heuristica' é a função que gera vizinhos (shift, swap, inversao).
def busca_local_primeira_melhoria(solucao, custo_original, heuristica, grafo):
    custo_atual = custo_original
    solucao_atual = solucao
    melhoria = True
    while melhoria:
        melhoria = False

        # Gera soluções vizinhas usando a heurística escolhida (shift, swap etc.)
        for nova_solucao in heuristica(solucao_atual):
            novo_custo = calcular_custo(nova_solucao, grafo)

            # Se a vizinha for melhor, atualiza e reinicia a busca
            if novo_custo < custo_atual:
                solucao_atual = nova_solucao
                custo_atual = novo_custo
                melhoria = True
                break

    return solucao_atual, custo_atual

#### Heurísticas

As heurísticas apresentadas funcionam como geradores (yield) responsáveis por produzir, uma a uma, novas soluções vizinhas a partir do caminho atual. Cada heurística recebe a solução vigente e cria pequenas variações estruturais que são avaliadas pela função de busca local `busca_local_primeira_melhoria`. Essas variações permitem que o algoritmo explore alternativas de percurso com potencial para reduzir o custo total.

Todas utilizam uma cópia da solução original, mantêm o primeiro e o último vértice fixos (origem e destino) e realizam alterações apenas na parte interna da solução, acessada por `range(1, len(solucao)-1)`.

##### Heurísticas Implementadas

* **Shift:** Move um vértice interno para outra posição do caminho, produzindo uma nova ordem parcial.

* **Swap:** Troca a posição entre dois vértices internos, criando pequenas permutações da solução.

* **Inversão:** Seleciona uma subsequência interna e inverte sua ordem, gerando modificações maiores, semelhantes ao movimento 2-opt.



In [ ]:
# Heurísticas que geram uma vizinhança (usam yield)
# Sempre preservam o primeiro e último vértice do caminho

def shift(solucao: list):
    # Move um vértice para outra posição
    for idx_vert_shift in range(1, len(solucao) - 1):
        for idx_destino_shift in range(2, len(solucao) - 1):
            # Cópia para não alterar a original
            solucao_cp = solucao[:]
            # Remove o vértice da posição atual
            vert_shift = solucao_cp.pop(idx_vert_shift)
            # Insere em outra posição
            solucao_cp.insert(idx_destino_shift, vert_shift)
            # Retorna nova solução
            yield solucao_cp


def swap(solucao: list):
    # Troca dois vértices de posição
    for idx_vert_swap in range(1, len(solucao) - 1):
        for idx_destino_swap in range(2, len(solucao) - 1):
            solucao_cp = solucao[:]
            # Troca simples entre duas posições
            solucao_cp[idx_vert_swap], solucao_cp[idx_destino_swap] = (
                solucao_cp[idx_destino_swap],
                solucao_cp[idx_vert_swap]
            )
            yield solucao_cp


def inversao(solucao: list):
    # Inverte um segmento interno do caminho
    for idx_inicio in range(1, len(solucao) - 1):
        for idx_fim in range(2, len(solucao) - 1):
            solucao_cp = solucao[:]
            # Segmento a inverter
            sublista = solucao_cp[idx_inicio:idx_fim]
            sublista.reverse()
            # Reconstrói a solução com o segmento invertido
            yield solucao_cp[:idx_inicio] + sublista + solucao_cp[idx_fim:]


## Busca por vizinho mais próximo

A função `*vizinho_mais_proximo*` implementa a heurística clássica do vizinho mais próximo para construir uma solução inicial. A ideia é sempre escolher, a partir do vértice atual, o vértice ainda não visitado que possui o menor custo na matriz do grafo. Dessa forma, ela gera um caminho completo de forma gulosa, visitando todos os vértices uma única vez e retornando ao início no final.

Na implementação, a função começa criando o conjunto de vértices não visitados e inicializando o percurso com o vértice de partida. Em cada iteração, ela busca dentro desse conjunto o vértice de menor custo em relação ao vértice atual, utilizando *idxmin* para localizar o índice correspondente na matriz. O percurso é atualizado acrescentando esse vizinho, o custo total é incrementado, e o vértice atual passa a ser o último escolhido. Quando todos os vértices são visitados, a função adiciona o custo de retorno ao vértice inicial, finalizando o ciclo Hamiltoniano.

O resultado retornado consiste na rota completa construída pela heurística e o custo total acumulado.

#### Exemplo:
Se o vértice 1 tem custo mínimo para o vértice 3, e em seguida o vértice 3 tem custo mínimo para o 2, o algoritmo seguirá essa sequência até visitar todos os nós.

In [ ]:
# Estamos considerando o label numerico
def vizinho_mais_proximo(grafo : pd.DataFrame, vertice_inicial : int | str):
    # Criando conjunto de vertices nao visitados
    # o conjunto contem os indices referentes ao vertice na matriz problema
    vertices_nao_visitados = set(grafo.index)
    percurso = [vertice_inicial]
    vertices_nao_visitados.remove(vertice_inicial - 1)
    dist_total = 0
    v_atual = vertice_inicial
    while len(vertices_nao_visitados) > 0:
        # Obtem o label (numero) do vertice com menor custo
        idx_vizinho_mais_proximo = grafo[v_atual].loc[list(vertices_nao_visitados)].idxmin(skipna=True)
        prox = idx_vizinho_mais_proximo + 1
        dist_total += grafo[v_atual][idx_vizinho_mais_proximo]
        percurso.append(int(prox))
        v_atual = prox
        vertices_nao_visitados.remove(idx_vizinho_mais_proximo)
    dist_total += grafo[v_atual][vertice_inicial -1]
    percurso.append(vertice_inicial)
    return percurso, dist_total

resultados_vmp = [None] * 12
resultados_vmp_busca_local = [None] * 12

In [ ]:
def aplicar_problema(grafo, inicial, heuristica_construtiva, heuristicas_busca_local, out=True):
    # Solução inicial via heurística construtiva
    caminho, custo = heuristica_construtiva(grafo, inicial)
    melhor_caminho, melhor_custo = caminho, custo
    heuristica_melhor_solucao = None

    if out:
        print(f"Aplicando Heurística Construtiva: {heuristica_construtiva.__name__}")
        print(f"Caminho inicial: {caminho}")
        print(f"Custo inicial: {custo}")
        print("\nAplicando Busca Local:")

    # Aplicação das heurísticas de busca local
    for heuristica in heuristicas_busca_local:
        solucao, custo_local = busca_local_primeira_melhoria(
            melhor_caminho, melhor_custo, heuristica, grafo
        )

        if out:
            print(f"\nHeurística: {heuristica.__name__}")
            print("Método: Primeira Melhoria")
            print(f"Caminho obtido: {solucao}")
            print(f"Custo obtido: {custo_local}")

        # Atualiza melhor solução
        if custo_local < melhor_custo:
            melhor_caminho, melhor_custo = solucao, custo_local
            heuristica_melhor_solucao = heuristica.__name__

    return melhor_caminho, melhor_custo, heuristica_melhor_solucao


### Problemas


#### Problema 1

Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em km.

In [ ]:
# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp, _ = vizinho_mais_proximo(matriz_problema_km, 1)
resultados_vmp[0] = caminho_vmp

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[0] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo inicial: 2291.3999999999996

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 48, 40, 36, 31, 7, 19, 34, 16, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 17, 25, 13, 24, 29, 43, 14, 46, 5, 41, 30, 6, 27, 44, 28, 15, 18, 45, 1]
Custo obtido: 2222.5999999999995

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 48, 40, 36, 25, 31, 7, 19, 34, 16, 33, 39, 35, 42, 26, 32, 17, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 41, 30, 6, 27, 28, 15, 18, 45, 44, 1]
Custo obtido: 2128.2

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 48, 40, 36, 25, 31, 7, 19, 

#### Problema 2
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pelo tempo em minutos.

In [ ]:
# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp, _ = vizinho_mais_proximo(matriz_problema_min, 1)
resultados_vmp[1] = caminho_vmp

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[1] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Custo inicial: 2355.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 19, 34, 16, 32, 26, 42, 35, 33, 20, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 27, 6, 30, 41, 47, 39, 1]
Custo obtido: 2291.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 48, 40, 25, 17, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 7, 31, 36, 22, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 27, 6, 30, 41, 5, 47, 38, 1]
Custo obtido: 2230.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 23, 41, 30, 6, 27, 44, 28, 15, 18, 45, 46, 14, 43, 29, 24, 5, 2, 11, 10, 12, 9, 8, 2

#### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [ ]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_36, custo_vmp_36 = vizinho_mais_proximo(matriz_problema_km_36, 1)
resultados_vmp[2] = caminho_vmp_36

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_36, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[2] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Custo inicial: 1951.4999999999998

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 36, 25, 31, 7, 34, 16, 33, 20, 17, 19, 32, 26, 35, 13, 24, 29, 14, 5, 18, 15, 28, 27, 6, 30, 1]
Custo obtido: 1781.9999999999998

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 36, 25, 31, 17, 7, 34, 16, 33, 20, 19, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 8, 1]
Custo obtido: 1772.8999999999996

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 21, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 8, 1]
Custo obtido: 1716.8999999999999

Melhor Escolha:

Melhor camin

#### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto em minutos




In [ ]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_36, custo_vmp_36 = vizinho_mais_proximo(matriz_problema_min_36, 1)
resultados_vmp[3] = caminho_vmp_36

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_36, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[3] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 7, 31, 36, 1]
Custo inicial: 1950.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 23, 4, 3, 22, 36, 25, 31, 7, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 21, 17, 20, 1]
Custo obtido: 1876.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 23, 4, 3, 22, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 21, 25, 17, 20, 1]
Custo obtido: 1859.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 21, 30, 6, 27, 28, 15, 18, 14, 29, 24, 13, 35, 33, 16, 34, 26, 32, 19, 7, 31, 36, 22, 25, 17, 20, 1]
Custo obtido: 1728.0

Melhor Escolha:

Melhor caminho final: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 

#### Problema 5
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_km_24 = matriz_problema_km[list(range(1,25))].iloc[0:24].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_24, custo_vmp_24 = vizinho_mais_proximo(matriz_problema_km_24, 1)
resultados_vmp[4] = caminho_vmp_24

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_24, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[4] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Custo inicial: 1379.3

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Custo obtido: 1352.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Custo obtido: 1337.3000000000004

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 2, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Custo obtido: 1328.6000000000001

Melhor Escolha:

Melhor caminho final: [1, 23, 2, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Melhor Custo:  1328.6000000000001
Heurística responsável: inversao


#### Problema 6
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_min_24 = matriz_problema_min[list(range(1,25))].iloc[0:24].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_24, custo_vmp_24 = vizinho_mais_proximo(matriz_problema_min_24, 1)
resultados_vmp[5] = caminho_vmp_24

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_24, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[5] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 20, 19, 16, 7, 13, 24, 14, 18, 15, 6, 1]
Custo inicial: 1283.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 1]
Custo obtido: 1255.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Custo obtido: 1242.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 2, 5, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Custo obtido: 1235.0

Melhor Escolha:

Melhor caminho final: [1, 23, 2, 5, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Melhor Custo:  1235.0
Heurística responsável: inversao


#### Problema 7
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_km_12 = matriz_problema_km[list(range(1,13))].iloc[0:12].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_12, custo_vmp_12 = vizinho_mais_proximo(matriz_problema_km_12, 1)
resultados_vmp[6] = caminho_vmp_12

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_12, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[6] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 10, 12, 11, 2, 3, 4, 5, 7, 6, 1]
Custo inicial: 828.4

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 6, 7, 5, 1]
Custo obtido: 772.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 10, 11, 2, 3, 4, 6, 12, 7, 5, 1]
Custo obtido: 745.7

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 6, 4, 3, 2, 11, 10, 8, 9, 12, 7, 5, 1]
Custo obtido: 707.9000000000001

Melhor Escolha:

Melhor caminho final: [1, 6, 4, 3, 2, 11, 10, 8, 9, 12, 7, 5, 1]
Melhor Custo:  707.9000000000001
Heurística responsável: inversao


#### Problema 8
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_min_12 = matriz_problema_min[list(range(1,13))].iloc[0:12].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_12, custo_vmp_12 = vizinho_mais_proximo(matriz_problema_min_12, 1)
resultados_vmp[7] = caminho_vmp_12

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_12, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[7] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 6, 7, 1]
Custo inicial: 687.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 8, 1]
Custo obtido: 649.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 1]
Custo obtido: 609.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 1]
Custo obtido: 609.0

Melhor Escolha:

Melhor caminho final: [1, 8, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 1]
Melhor Custo:  609.0
Heurística responsável: shift


#### Problema 9
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em km

In [ ]:
# Seleção das cidades específicas
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

# Criação da submatriz
matriz_problema_km_7 = matriz_problema_km[cidades_7].iloc[indices_7].copy()
matriz_problema_km_7.index = range(len(cidades_7))
matriz_problema_km_7.columns = range(1, len(cidades_7) + 1)

# Caminho inicial (Vizinho Mais Próximo)
caminho_inicial, custo_inicial = vizinho_mais_proximo(matriz_problema_km_7, 1)
resultados_vmp[8] = caminho_inicial

# Aplicando busca local com as heurísticas
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_7,1,vizinho_mais_proximo,[swap, shift, inversao])
resultados_vmp_busca_local[8] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 3, 4, 5, 7, 6, 2, 1]
Custo inicial: 518.5

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Melhor Escolha:

Melhor caminho final: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor Custo:  438.3
Heurística responsável: swap


#### Problema 10
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em minutos


In [ ]:
# Seleção das cidades específicas
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

# Criação da submatriz
matriz_problema_min_7 = matriz_problema_min[cidades_7].iloc[indices_7].copy()
matriz_problema_min_7.index = range(len(cidades_7))
matriz_problema_min_7.columns = range(1, len(cidades_7) + 1)

# Caminho inicial (Vizinho Mais Próximo)
caminho_inicial, custo_inicial = vizinho_mais_proximo(matriz_problema_min_7, 1)
resultados_vmp[9] = caminho_inicial

# Aplicando busca local com as heurísticas
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_7,1,vizinho_mais_proximo,[swap, shift, inversao])
resultados_vmp_busca_local[9] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 3, 4, 7, 5, 6, 2, 1]
Custo inicial: 413.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Melhor Escolha:

Melhor caminho final: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor Custo:  364.0
Heurística responsável: swap


#### Problema 11
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo a distância em km


In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_km_6 = matriz_problema_km[list(range(1,7))].iloc[0:6].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_inicial, custo_inicial = vizinho_mais_proximo(matriz_problema_km_6, 1)
resultados_vmp[10] = caminho_inicial

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_6, 1, vizinho_mais_proximo,[swap, shift, inversao])
resultados_vmp_busca_local[10] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 3, 4, 2, 5, 6, 1]
Custo inicial: 463.6

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 348.3

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 348.3

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 348.3

Melhor Escolha:

Melhor caminho final: [1, 5, 2, 3, 4, 6, 1]
Melhor Custo:  348.3
Heurística responsável: swap


#### Problema 12
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos


In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_min_6 = matriz_problema_min[list(range(1,7))].iloc[0:6].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_inicial, custo_inicial = vizinho_mais_proximo(matriz_problema_min_6, 1)
resultados_vmp[11] = caminho_inicial

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_6, 1, vizinho_mais_proximo,[swap, shift, inversao])
resultados_vmp_busca_local[11] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 3, 4, 2, 5, 6, 1]
Custo inicial: 400.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 305.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 305.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 305.0

Melhor Escolha:

Melhor caminho final: [1, 5, 2, 3, 4, 6, 1]
Melhor Custo:  305.0
Heurística responsável: swap


## Inserção mais barata

A função `insercao_mais_barata` constrói uma rota inicial usando a ideia de sempre inserir o próximo vértice no ponto onde ele aumenta o custo o mínimo possível. Primeiro ela monta um ciclo simples com o vértice inicial e o vértice mais próximo dele. Depois, enquanto ainda houver vértices de fora, escolhe sempre o vértice mais próximo de qualquer vértice já presente na rota.

Para cada vértice escolhido, o algoritmo testa todas as posições possíveis dentro da rota e calcula quanto o custo aumentaria caso ele fosse inserido ali. A posição que causa o menor aumento é usada. Isso continua até que todos os vértices tenham sido incluídos na rota.

No final, a função retorna a rota construída e o custo total calculado.

In [ ]:
# Estamos considerando o label numerico
def insercao_mais_barata(grafo : pd.DataFrame, vertice_inicial : int | str):

    vertices_restantes = set(grafo.index)

    # Selecionamos o indice do vertice mais proximo a raiz de forma gulosa
    v_mais_proximo = int(grafo[vertice_inicial].idxmin())

    # Ciclo inicial
    rota = [vertice_inicial, v_mais_proximo + 1, vertice_inicial]
    vertices_restantes.remove( vertice_inicial - 1, )
    vertices_restantes.remove(v_mais_proximo)
    vertices_inseridos = {vertice_inicial - 1, v_mais_proximo}

    custo_total = 0
    while vertices_restantes:
        # Estamos selecionando o vertice de 'vertices_restantes' que tem a menor distancia a outro vertice
        # Filtra apenas as linhas da matriz referentes aos vertices do conjunto de vertices_restantes
        r = int(grafo[list(vertices_restantes)].iloc[list(vertices_inseridos)].min().idxmin())

        # Consegue o indice do vertice do conjunto 'vertices_restantes' que tem a menor distancia
        # ao algum vertice incluido na rota. Para manter este codigo similar ao pseudocodigo visto em sala, chamaremos a variavel de
        # Encontrar a melhor posicao para inserir a rota
        melhor_custo_insercao = math.inf
        melhor_posicao_insercao = None
        for i in range(1, len(rota) - 1):
            u, v = rota[i] - 1, rota[i + 1] - 1
            custo = grafo[u + 1][r] + grafo[r + 1][v] - grafo[u + 1][v]
            if custo < melhor_custo_insercao:
                melhor_custo_insercao = custo
                melhor_posicao_insercao = i

        rota.insert(melhor_posicao_insercao + 1, r + 1)
        vertices_restantes.remove(r)
        vertices_inseridos.add(r)
        custo_total += melhor_custo_insercao
    return rota, calcular_custo(rota, grafo)

resultados_imb = [None] * 12
resultados_imb_busca_local = [None] * 12


### Problemas

#### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km

In [ ]:
# Caminho inicial gerado pela heurística de Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km, 1)
resultados_imb[0] = caminho_imb

# Busca local usando swap, shift e inversão após Inserção Mais Barata
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km,  1, insercao_mais_barata, [swap, shift, inversao])
resultados_imb_busca_local[0] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo:", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Custo inicial: 2148.1

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 21, 9, 11, 10, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Custo obtido: 2115.5999999999995

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 21, 9, 10, 12, 22, 48, 40, 36, 25, 31, 17, 7, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 11, 2, 23, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Custo obtido: 2078.2999999999997

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 10, 12, 9, 8, 21, 22, 48, 40, 36, 25, 31, 17, 7, 19, 34, 16, 32, 2

#### Problema 2

Percurso por 48 cidades, partindo de ANGICOS, com função custo definida pelo tempo

In [ ]:
# Caminho inicial gerado pela heurística de Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min, 1)
resultados_imb[1] = caminho_imb

# Busca local usando swap, shift e inversão após Inserção Mais Barata
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min,  1, insercao_mais_barata, [swap, shift, inversao])
resultados_imb_busca_local[1] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo:", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 33, 39, 35, 42, 32, 26, 16, 34, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 11, 5, 2, 3, 4, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1]
Custo inicial: 2197.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 39, 33, 35, 42, 26, 32, 16, 34, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 11, 5, 2, 4, 3, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1]
Custo obtido: 2167.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 39, 35, 42, 26, 32, 16, 34, 33, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 3, 4, 2, 11, 5, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1]
Custo obtido: 2119.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 38, 20, 17, 25, 40, 48, 22, 36, 31, 7, 19, 33, 34, 16, 32, 26, 42, 35, 39, 4

#### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [ ]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_36, 1)
resultados_imb[2] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_36,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[2] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 21, 9, 12, 10, 11, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1]
Custo inicial: 1835.3

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 21, 9, 11, 10, 12, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1]
Custo obtido: 1814.8999999999999

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 21, 9, 10, 12, 22, 36, 25, 31, 17, 7, 19, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 11, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1]
Custo obtido: 1786.6000000000001

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 10, 12, 9, 8, 21, 22, 36, 25, 31, 17, 7, 19, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 4, 3, 2, 11, 5, 23, 1]
Custo obtido: 1743.2

Melhor Escolha:

Melhor caminho: [1, 10, 12, 9, 8, 21

#### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto em minutos

In [ ]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_36, 1)
resultados_imb[3] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_36,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[3] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 12, 29, 24, 13, 20, 17, 25, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 22, 10, 11, 5, 14, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Custo inicial: 1931.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 29, 24, 13, 20, 17, 25, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 22, 10, 11, 5, 14, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Custo obtido: 1931.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 14, 29, 24, 13, 20, 17, 25, 22, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Custo obtido: 1828.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 20, 17, 25, 22, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 13, 24, 29, 14, 5, 11, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Custo obtido: 1765.0

Melhor Escolha:

Melhor caminho: [1, 8, 9, 12, 10, 20, 17, 25, 22, 36, 31, 7,

#### Problema 5
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_km_24 = matriz_problema_km[list(range(1,25))].iloc[0:24].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_24, 1)
resultados_imb[4] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_24,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[4] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 12, 10, 11, 21, 22, 17, 7, 16, 19, 20, 13, 24, 14, 5, 2, 3, 4, 6, 15, 18, 23, 1]
Custo inicial: 1459.3000000000002

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 2, 3, 4, 18, 15, 6, 23, 1]
Custo obtido: 1405.1000000000001

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 11, 2, 3, 4, 18, 15, 6, 23, 1]
Custo obtido: 1390.5000000000002

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 11, 2, 3, 4, 18, 15, 6, 23, 1]
Custo obtido: 1390.5000000000002

Melhor Escolha:

Melhor caminho: [1, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 11, 2, 3, 4, 18, 15, 6, 23, 1]
Melhor custo: 1390.5000000000002
Heurística: shift


#### Problema 6
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_min_24 = matriz_problema_min[list(range(1,25))].iloc[0:24].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_24, 1)
resultados_imb[5] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_24,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[5] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 12, 24, 13, 20, 19, 16, 7, 17, 22, 21, 23, 10, 11, 5, 14, 2, 3, 4, 6, 15, 18, 1]
Custo inicial: 1355.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 10, 11, 5, 14, 2, 3, 4, 6, 15, 18, 1]
Custo obtido: 1351.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 2, 3, 4, 6, 15, 18, 14, 5, 11, 10, 1]
Custo obtido: 1260.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 2, 3, 4, 6, 15, 18, 14, 5, 11, 10, 1]
Custo obtido: 1260.0

Melhor Escolha:

Melhor caminho: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 2, 3, 4, 6, 15, 18, 14, 5, 11, 10, 1]
Melhor custo: 1260.0
Heurística: shift


#### Problema 7
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_km_12 = matriz_problema_km[list(range(1,13))].iloc[0:12].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_12, 1)
resultados_imb[6] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_12,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[6] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo inicial: 711.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 711.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 711.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 11, 10, 12, 7, 5, 2, 3, 4, 6, 1]
Custo obtido: 699.2

Melhor Escolha:

Melhor caminho: [1, 8, 9, 11, 10, 12, 7, 5, 2, 3, 4, 6, 1]
Melhor custo: 699.2
Heurística: inversao


#### Problema 8
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_min_12 = matriz_problema_min[list(range(1,13))].iloc[0:12].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_12, 1)
resultados_imb[7] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_12,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[7] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo inicial: 609.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 609.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 609.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 609.0

Melhor Escolha:

Melhor caminho: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo: 609.0
Heurística: None


#### Problema 9
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em km


In [ ]:
# cidades_7 = [1, 7, 8, 9, 10, 11, 12]
# indices_7 = [c - 1 for c in cidades_7]

# sub_matriz = matriz_problema_km[cidades_7].iloc[indices_7].copy()
# mapa_traducao = { i + 1: id_real for i, id_real in enumerate(cidades_7) }

# matriz_normalizada = sub_matriz.reset_index(drop=True)
# matriz_normalizada.columns = range(1, len(matriz_normalizada) + 1)

# melhor_caminho_ficticio, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
#     matriz_normalizada,
#     1,
#     insercao_mais_barata,
#     [swap, shift, inversao]
# )

# melhor_caminho_real = [mapa_traducao[cidade_ficticia] for cidade_ficticia in melhor_caminho_ficticio]

# print(f"\nMelhor caminho: {melhor_caminho_real}")
# print(f"Melhor custo: {melhor_custo}")
# print(f"Heurística: {heuristica_melhor_solucao}")

# resultados_imb[8] = [mapa_traducao[cidade_ficticia] for cidade_ficticia in insercao_mais_barata(matriz_normalizada, 1)[0]]
# resultados_imb_busca_local[8] = melhor_caminho_real

# Seleção das cidades [1, 7, 8, 9, 10, 11, 12]

cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

# Filtra a matriz de acordo com as cidades selecionadas
matriz_problema_km_7 = matriz_problema_km[cidades_7].iloc[indices_7].copy()

# Ajusta índices e colunas para valores sequenciais
matriz_problema_km_7.index = range(len(cidades_7))
matriz_problema_km_7.columns = range(1, len(cidades_7) + 1)

# Caminho inicial pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_7, 1)
resultados_imb[8] = caminho_imb

# Busca local com swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_7, 1, insercao_mais_barata, [swap, shift, inversao])

# Armazena resultados da busca local
resultados_imb_busca_local[8] = melhor_caminho

print("\nMelhor Escolha:")
print("Melhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)



Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 3, 4, 2, 7, 5, 6, 1]
Custo inicial: 438.3

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Melhor Escolha:
Melhor caminho: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo: 438.3
Heurística: None


#### Problema 10
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em minutos


In [ ]:
# cidades_7 = [1, 7, 8, 9, 10, 11, 12]
# indices_7 = [c - 1 for c in cidades_7]

# sub_matriz = matriz_problema_min[cidades_7].iloc[indices_7].copy()

# mapa_traducao = { i + 1: id_real for i, id_real in enumerate(cidades_7) }

# matriz_normalizada = sub_matriz.reset_index(drop=True)
# matriz_normalizada.columns = range(1, len(matriz_normalizada) + 1)

# melhor_caminho_ficticio, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
#     matriz_normalizada,
#     1,
#     insercao_mais_barata,
#     [swap, shift, inversao]
# )

# melhor_caminho_real = [mapa_traducao[cidade_ficticia] for cidade_ficticia in melhor_caminho_ficticio]

# print(f"\nMelhor caminho (Original): {melhor_caminho_real}")
# print(f"Melhor custo: {melhor_custo}")
# print(f"Heurística vencedora: {heuristica_melhor_solucao}")

# resultados_imb[9] = [mapa_traducao[cidade_ficticia] for cidade_ficticia in insercao_mais_barata(matriz_normalizada, 1)[0]]
# resultados_imb_busca_local[9] = melhor_caminho_real

# Seleção das cidades [1, 7, 8, 9, 10, 11, 12]
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

# Filtra a matriz de acordo com as cidades selecionadas
matriz_problema_min_7 = matriz_problema_min[cidades_7].iloc[indices_7].copy()

# Ajusta índices e colunas para valores sequenciais
matriz_problema_min_7.index = range(len(cidades_7))
matriz_problema_min_7.columns = range(1, len(cidades_7) + 1)

# Caminho inicial pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_7, 1)
resultados_imb[9] = caminho_imb

# Busca local com swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
    matriz_problema_min_7,
    1,
    insercao_mais_barata,
    [swap, shift, inversao]
)

# Armazena resultados da busca local
resultados_imb_busca_local[9] = melhor_caminho

print("\nMelhor Escolha:")
print("Melhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 3, 4, 2, 7, 5, 6, 1]
Custo inicial: 364.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Melhor Escolha:
Melhor caminho: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo: 364.0
Heurística: None


#### Problema 11
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo a distância em km


In [ ]:
# Filtra as colunas e linhas para as cidades [1, 6]
matriz_problema_km_6 = matriz_problema_km[list(range(1, 7))].iloc[0:6].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_6, 1)
resultados_imb[10] = caminho_imb

# Busca local com swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_6, 1, insercao_mais_barata, [swap, shift, inversao])
resultados_imb_busca_local[10] = melhor_caminho

print("\nMelhor Escolha:")
print("Melhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 3, 4, 6, 2, 5, 1]
Custo inicial: 348.5

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 6, 4, 3, 2, 5, 1]
Custo obtido: 348.3

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 6, 4, 3, 2, 5, 1]
Custo obtido: 348.3

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 6, 4, 3, 2, 5, 1]
Custo obtido: 348.3

Melhor Escolha:
Melhor caminho: [1, 6, 4, 3, 2, 5, 1]
Melhor custo: 348.3
Heurística: swap


#### Problema 12
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [ ]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_min_6 = matriz_problema_min[list(range(1, 7))].iloc[0:6].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_6, 1)
resultados_imb[11] = caminho_imb

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_6,  1,  insercao_mais_barata,  [swap, shift, inversao])
resultados_imb_busca_local[11] = melhor_caminho

print("\nMelhor Escolha:")
print("Melhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 3, 4, 6, 2, 5, 1]
Custo inicial: 305.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 6, 2, 5, 1]
Custo obtido: 305.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 6, 2, 5, 1]
Custo obtido: 305.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 6, 2, 5, 1]
Custo obtido: 305.0

Melhor Escolha:
Melhor caminho: [1, 3, 4, 6, 2, 5, 1]
Melhor custo: 305.0
Heurística: None


## Genético

Um algoritmo genético é uma técnica de otimização inspirada na evolução natural, usada para encontrar boas soluções para problemas complexos, como roteamento, planejamento ou design de sistemas. Ele simula o processo de seleção natural, onde as soluções mais “aptas” têm mais chance de sobreviver e gerar descendentes melhores.

O funcionamento do algoritmo genético envolve algumas etapas principais:

* **População inicial:** Cria-se um conjunto de soluções possíveis (chamadas de indivíduos ou cromossomos). Essas soluções podem ser geradas aleatoriamente ou usando algum critério heurístico.

* **Avaliação:** Cada indivíduo é avaliado de acordo com uma função de custo ou qualidade, que mede quão boa é a solução.

* **Seleção:** Escolhem-se os melhores indivíduos para se tornarem “pais” da próxima geração. Técnicas comuns incluem torneio, roleta e elitismo.

* **Cruzamento:** Os pais combinam partes de suas soluções para gerar descendentes, tentando preservar características boas de cada um.

* **Mutação:** Pequenas alterações são aplicadas nos descendentes para explorar novas soluções e evitar que o algoritmo fique preso em mínimos locais.

* **Renovação:** Determina quais indivíduos entram na próxima geração, mantendo o tamanho da população constante e garantindo que boas soluções não se percam.

O processo se repete por várias gerações até que se alcance uma solução satisfatória ou que o número máximo de iterações seja atingido.

### Gerador de soluções aleatórias

A função `solucao_aleatoria` cria uma rota válida para o problema de roteamento, garantindo que todos os vértices do grafo sejam visitados exatamente uma vez antes de retornar ao ponto inicial. Primeiro, ela cria uma lista com todos os vértices, representados pelos índices do DataFrame, e remove o vértice inicial, que será fixo no início e no fim da rota. Em seguida, embaralha os vértices restantes de forma aleatória, gerando uma permutação diferente a cada chamada. Por fim, concatena o vértice inicial no começo e no fim.

A função `calcular_custo` percorre a rota gerada e soma os custos das arestas entre cada par de vértices consecutivos. Cada valor na matriz do grafo representa o custo de ir de um vértice a outro, e a soma total desses custos fornece o custo completo da solução.


In [ ]:
import random
def solucao_aleatoria(grafo : pd.DataFrame, vertice_inicial):
    rota = list(map(lambda i : i + 1,grafo.index))
    rota.remove(vertice_inicial)
    random.shuffle(rota)
    return [vertice_inicial] + rota + [vertice_inicial]

# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        # print("v1: ", v1)
        # print("v2: ", v2)
        custo += grafo[v1][v2 - 1]
    return custo

samples = [solucao_aleatoria(matriz_problema_km, 1) for i in range(0,6)]
print(samples)
# custos = [calcular_custo(solucao, matriz_problema_km) for solucao in samples]
# print(f"Minimo : {min(custos)}")
# print(f"Maximo : {max(custos)}")


[[1, 29, 42, 41, 7, 36, 11, 10, 33, 18, 14, 34, 40, 37, 6, 12, 22, 45, 27, 24, 19, 23, 31, 13, 5, 26, 32, 43, 20, 15, 3, 46, 48, 16, 35, 4, 30, 47, 39, 44, 2, 17, 25, 8, 38, 28, 9, 21, 1], [1, 30, 13, 41, 36, 39, 27, 7, 33, 21, 45, 10, 11, 16, 15, 29, 8, 2, 23, 48, 3, 38, 25, 5, 32, 6, 37, 22, 26, 19, 12, 31, 44, 35, 42, 40, 46, 18, 34, 20, 4, 43, 14, 24, 47, 28, 9, 17, 1], [1, 35, 39, 6, 26, 24, 25, 40, 5, 9, 37, 43, 4, 16, 22, 12, 17, 3, 8, 44, 23, 29, 15, 48, 41, 18, 10, 2, 13, 34, 30, 42, 21, 45, 36, 27, 14, 7, 47, 46, 33, 28, 20, 11, 31, 19, 38, 32, 1], [1, 6, 7, 14, 12, 43, 16, 48, 19, 45, 2, 4, 33, 5, 37, 26, 20, 9, 41, 13, 46, 32, 35, 36, 28, 10, 24, 38, 21, 23, 47, 31, 30, 17, 25, 11, 44, 42, 27, 39, 34, 40, 29, 3, 22, 18, 8, 15, 1], [1, 20, 18, 4, 10, 32, 29, 22, 40, 9, 25, 16, 26, 8, 13, 30, 34, 27, 2, 33, 31, 12, 35, 7, 37, 48, 3, 28, 19, 39, 6, 11, 46, 43, 14, 23, 44, 17, 15, 42, 24, 36, 38, 47, 41, 45, 21, 5, 1], [1, 18, 42, 4, 39, 16, 10, 24, 14, 8, 17, 12, 7, 30, 37, 33

### Populações


Como explicamos acima o conceito de população,agora iremos gerar as populações com 15 indivíduos

In [ ]:
def criar_populacao(grafo, resultados_imb, resultados_vmp, resultados_imb_busca_local, resultados_vmp_busca_local, indice, tamanho_aleatorio=36, vertice_inicial=1):

    # Seleciona as soluções heurísticas para este problema
    solucao_insercao_mais_barata = resultados_imb[indice]
    solucao_vizinho_mais_proximo = resultados_vmp[indice]
    solucao_insercao_mais_barata_local = resultados_imb_busca_local[indice]
    solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[indice]

    # Gera soluções aleatórias
    solucoes_aleatorias = [solucao_aleatoria(grafo, vertice_inicial) for _ in range(tamanho_aleatorio)]

    # Combina todas as soluções
    populacao = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local ]

    # Ordena pelo custo
    populacao = sorted(populacao, key=lambda sol: calcular_custo(sol, grafo))

    print(f"População problema {indice+1} criada. Tamanho: {len(populacao)} | Melhor custo: {calcular_custo(populacao[0], grafo):.2f}")

    return populacao

In [ ]:
pop_problema_1  = criar_populacao(matriz_problema_km,      resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 0)
pop_problema_2  = criar_populacao(matriz_problema_min,     resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 1)
pop_problema_3  = criar_populacao(matriz_problema_km_36,   resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 2)
pop_problema_4  = criar_populacao(matriz_problema_min_36,  resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 3)
pop_problema_5  = criar_populacao(matriz_problema_km_24,   resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 4)
pop_problema_6  = criar_populacao(matriz_problema_min_24,  resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 5)
pop_problema_7  = criar_populacao(matriz_problema_km_12,   resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 6)
pop_problema_8  = criar_populacao(matriz_problema_min_12,  resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 7)
pop_problema_9  = criar_populacao(matriz_problema_km_7,    resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 8)
pop_problema_10 = criar_populacao(matriz_problema_min_7,   resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 9)
pop_problema_11 = criar_populacao(matriz_problema_km_6,    resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 10)
pop_problema_12 = criar_populacao(matriz_problema_min_6,   resultados_imb, resultados_vmp,
                                           resultados_imb_busca_local, resultados_vmp_busca_local, 11)

População problema 1 criada. Tamanho: 40 | Melhor custo: 2046.20
População problema 2 criada. Tamanho: 40 | Melhor custo: 2107.00
População problema 3 criada. Tamanho: 40 | Melhor custo: 1716.90
População problema 4 criada. Tamanho: 40 | Melhor custo: 1728.00
População problema 5 criada. Tamanho: 40 | Melhor custo: 1328.60
População problema 6 criada. Tamanho: 40 | Melhor custo: 1235.00
População problema 7 criada. Tamanho: 40 | Melhor custo: 699.20
População problema 8 criada. Tamanho: 40 | Melhor custo: 609.00
População problema 9 criada. Tamanho: 40 | Melhor custo: 438.30
População problema 10 criada. Tamanho: 40 | Melhor custo: 364.00
População problema 11 criada. Tamanho: 40 | Melhor custo: 348.30
População problema 12 criada. Tamanho: 40 | Melhor custo: 305.00


### Funções para o algoritmo Genético

#### Representação dos cromossomos


Transformamos cada população em uma lista de cromossomos com seus custos para que todos os operadores genéticos possam avaliar a qualidade da solução, facilitando seleção, cruzamento e mutação de forma consistente.

In [ ]:
# Para cada população de soluções, calculamos o custo de cada rota
# Cada cromossomo é representado como uma tupla (solucao, custo)
cromossomos_problema1 = [(solucao, calcular_custo(solucao, matriz_problema_km)) for solucao in pop_problema_1]
cromossomos_problema2 = [(solucao, calcular_custo(solucao, matriz_problema_min)) for solucao in pop_problema_2]
cromossomos_problema3 = [(solucao, calcular_custo(solucao, matriz_problema_km_36)) for solucao in pop_problema_3]
cromossomos_problema4 = [(solucao, calcular_custo(solucao, matriz_problema_min_36)) for solucao in pop_problema_4]
cromossomos_problema5 = [(solucao, calcular_custo(solucao, matriz_problema_km_24)) for solucao in pop_problema_5]
cromossomos_problema6 = [(solucao, calcular_custo(solucao, matriz_problema_min_24)) for solucao in pop_problema_6]
cromossomos_problema7 = [(solucao, calcular_custo(solucao, matriz_problema_km_12)) for solucao in pop_problema_7]
cromossomos_problema8 = [(solucao, calcular_custo(solucao, matriz_problema_min_12)) for solucao in pop_problema_8]
cromossomos_problema9 = [(solucao, calcular_custo(solucao, matriz_problema_km_7)) for solucao in pop_problema_9]
cromossomos_problema10 = [(solucao, calcular_custo(solucao, matriz_problema_min_7)) for solucao in pop_problema_10]
cromossomos_problema11 = [(solucao, calcular_custo(solucao, matriz_problema_km_6)) for solucao in pop_problema_11]
cromossomos_problema12 = [(solucao, calcular_custo(solucao, matriz_problema_min_6)) for solucao in pop_problema_12]

#### Seleção dos genitores

Seleciona os melhores cromossomos da população e combina cada um com um genitor aleatório, garantindo que soluções de baixo custo sobrevivam enquanto mantém diversidade genética.

In [ ]:
def elitismo(populacao : list, num_pares):
  # Ordena a população pelo custo (menor primeiro)
  populacao_ordenada = sorted(populacao, key=lambda x: x[1])
  genitores = []

  # Seleciona os pares de genitores: o melhor sempre participa
  for i in range(0,num_pares):
      # Escolhe o segundo genitor aleatoriamente para manter diversidade
      segundo_genitor = random.choice(range(0, num_pares))
      if segundo_genitor == i:
          segundo_genitor = 0
      genitores.append((populacao_ordenada[i], populacao_ordenada[segundo_genitor]))
  return genitores


Escolhe genitores de forma competitiva em torneios aleatórios, selecionando os melhores dentro de cada subconjunto, equilibrando pressão seletiva e diversidade genética.

In [ ]:
def torneio(populacao: list, num_pares: int, tamanho_torneio: int = 3):
  genitores = []

  # Para cada par a ser gerado
  for _ in range(num_pares):
    # Seleciona competidores aleatórios e escolhe o de menor custo
    competidores_1 = random.sample(populacao, k=tamanho_torneio)
    pai1 = min(competidores_1, key=lambda x: x[1])
    competidores_2 = random.sample(populacao, k=tamanho_torneio)
    pai2 = min(competidores_2, key=lambda x: x[1])

    # Adiciona o par à lista de genitores
    genitores.append((pai1, pai2))
  return genitores

#### Operadores de crossover

A função `crossover_1_ponto` divide os genitores ao meio e combina as metades, corrigindo os que se duplicam e garantindo que cada rota gerada seja válida, mantendo a integridade da solução para problemas de roteamento.

In [ ]:
def crossover_1_ponto(cromossomo_1, cromossomo_2):
    n = len(cromossomo_1) # Cromossomos tem comprimentos iguais
    # Ponto de corte
    mid = n // 2
    offspring_1 = cromossomo_1[0:mid] + cromossomo_2[mid:n]
    offspring_2 = cromossomo_2[0:mid] + cromossomo_1[mid:n]
    offsprings = [offspring_1, offspring_2]
    # As labels sao numeros em sequencia
    vertices = set(range(2, n))
    for i, offspring in enumerate(offsprings):
        # vertices ja presentes no offspring atual
        vertices_no_offspring = set(offspring)
        vertices_no_offspring.remove(1)
        # se o numero de elementos unicos eh menor que o tamanho - 1 (ja que eh um ciclo),
        # ja elementos repetidos
        if len(vertices_no_offspring) < n-1:
            vertices_restantes = vertices.difference(vertices_no_offspring)
            novo_offspring = offspring[0:mid]
            controle = set(offspring[1:mid])
            for v in offspring[mid:]:
                novo_v = None
                if v in controle:
                    novo_v = vertices_no_offspring.pop()
                else:
                    novo_v = v
                controle.add(novo_v)
                novo_offspring.append(novo_v)
            offsprings[i] = novo_offspring
    return offsprings[0], offsprings[1]


A função `crossover_ox` gera dois descendentes combinando partes de dois genitores, preservando a ordem e garantindo que cada vértice apareça apenas uma vez.

In [ ]:
def crossover_ox(genitor_1, genitor_2):
  # Escolhe um ponto de corte aleatório entre 1 e penúltimo índice
  ponto = random.choice(range(1, len(genitor_1) - 1))
  # Cria os dois descendentes pegando os segmentos iniciais de cada genitor
  offspring_1, offspring_2 = genitor_1[:ponto], genitor_2[:ponto]

  vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)

  # Preenche o restante do offspring_1 mantendo a ordem de genitor_2
  for i in genitor_2:
      if i not in vertices_incluidos1:
          offspring_1.append(i)
  offspring_1.append(genitor_1[0])

  # Preenche o restante do offspring_2 mantendo a ordem de genitor_1
  for i in genitor_1:
      if i not in vertices_incluidos2:
          offspring_2.append(i)
  offspring_2.append(genitor_2[0])
  return offspring_1, offspring_2


A função `crossover_uniforme` mistura genes dos dois genitores posição a posição, garantindo diversidade genética e mantendo rotas válidas, ideal para introduzir novas combinações sem perder a viabilidade do ciclo

In [ ]:
def crossover_uniforme(genitor_1,genitor_2):
    # Inicializa os descendentes como cópias dos genitores
    offspring_1, offspring_2 = genitor_1, genitor_2
    # Conjuntos de genes que ainda podem ser usados (vértices disponíveis)
    genes_faltantes_1, genes_faltantes_2 = set(range(0, 49)), set(range(0, 49))

    # Percorre cada posição (exceto início e fim) e decide aleatoriamente de qual genitor o gene virá
    for i in range(1, len(offspring_1) - 1):
        if random.random() < 0.5:
            if offspring_2[i] not in genes_faltantes_1:
                offspring_1[i] = genes_faltantes_1.pop()
            else:
                offspring_1[i] = offspring_2[i]
            genes_faltantes_1.add(offspring_1[i])
        if random.random() < 0.5:
            if offspring_1[i] not in genes_faltantes_2:
                offspring_2[i] = genes_faltantes_2.pop()
            else:
                offspring_2[i] = offspring_1[i]
            genes_faltantes_2.add(offspring_2[i])
    return offspring_1, offspring_2

A função de `cruzamento` aplica o crossover em todos os pares de genitores, cria descendentes e calcula o custo de cada rota. Garante que os novos cromossomos estejam prontos para as próximas etapas do algoritmo genético.

In [ ]:
def cruzamento(genitores, grafo):
  # Lista para armazenar todos os descendentes gerados
    offsprings = []
    for g in genitores:
        genitor_1,genitor_2=g
        offspring1, offspring2 = crossover_ox(genitor_1[0], genitor_2[0])
        # print(f"Genitores: {genitor_1[0]} e {genitor_2[0]}")
        # print(f"Offsprings: {offspring1[0]} e {offspring2[0]}")
        # Adiciona os descendentes à lista junto com seu custo calculado usando a matriz do grafo
        offsprings.extend([(offspring1, calcular_custo(offspring1, grafo)), (offspring2,calcular_custo(offspring2, grafo))])
    return offsprings

In [ ]:
## Order Crossover (ANTIGO)
# def crossover_ox(genitor_1, genitor_2):
#     offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
#     vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
#     for i in genitor_2:
#         if i not in vertices_incluidos1:
#             offspring_1.append(i)
#     offspring_1.append(genitor_1[0])
#     for i in genitor_1:
#         if i not in vertices_incluidos2:
#             offspring_2.append(i)
#     offspring_2.append(genitor_2[0])
#     return offspring_1, offspring_2
## Partially-Bmapped Crossover (à finalizar)
# https://www.nature.com/articles/s41598-025-86695-4
# def crossover_pbx(genitor_1, genitor_2):
#     posicoes = random.randint(1, len(genitor_1) - 1)
#     pos_genes_1 = set(random.choices(range(1, len(genitor_1) - 1), k=posicoes))
#     pos_genes_2 = set(range(0,48)).difference(genes_1)
#     genes_1 = {genitor_1[i] for i in pos_genes_1}
#     genes_2 = {}
#     offspring_1, offspring_2 = [0 for i in range(48)], [0 for i in range(48)]
#     for i in genes:
#         offspring_1[i] = genitor_1[i]
#     for i in range(0,48):
#         if i in pos_genes_1:
#             continue
#         if genitor_2[i] in genes_1:
#             g = genes_2.pop()
#             offspring_1[i] = g
#     genes_1 = random.choices(range(1, len(genitor_1) - 1), k=posicoes)
#     offspring_1, offspring_2 = genitor_1[:ponto], genitor_2[:ponto]
#     vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
#     for i in genitor_2:
#         if i not in vertices_incluidos1:
#             offspring_1.append(i)
#     offspring_1.append(genitor_1[0])
#     for i in genitor_1:
#         if i not in vertices_incluidos2:
#             offspring_2.append(i)
#     offspring_2.append(genitor_2[0])
#     return offspring_1, offspring_2

# def crossover_2_pontos(genitor_1, genitor_2):
#     offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
#     vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
#     g1, g2 = random.sample(range(1,len(genitor_1) - 1), 2)
#     g_1_inicio, g_1_final = min(g1,g2), max(g1,g2)
#     g3, g4 = random.sample(range(1,len(genitor_1) - 1), 2)
#     g_2_inicio, g_2_final = min(g1,g2), max(g1,g2)
#     for i in genitor_2:
#         if i not in vertices_incluidos1:
#             offspring_1.append(i)
#     offspring_1.append(genitor_1[0])
#     for i in genitor_1:
#         if i not in vertices_incluidos2:
#             offspring_2.append(i)
#     offspring_2.append(genitor_2[0])
#     return offspring_1, offspring_2
#     pass

# def crossover_2_pontos(genitor_1, genitor_2):
#     offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
#     vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
#     g1, g2 = random.sample(range(1,len(genitor_1) - 1), 2)
#     g_1_inicio, g_1_final = min(g1,g2), max(g1,g2)
#     g3, g4 = random.sample(range(1,len(genitor_1) - 1), 2)
#     g_2_inicio, g_2_final = min(g1,g2), max(g1,g2)
#     segmento_1 = genitor_1[g1:g2]
#     vertices_segmento_1 = set(segmento_1)
#     vertices_segmento_2 = set(segmento_2)
#     for v in segmento_2:
#         if v in segmento_1:
#
#     segmento_2 = genitor_2[g3:g4]
#
#
#     for i in genitor_2:
#         if i not in vertices_incluidos1:
#             offspring_1.append(i)
#     offspring_1.append(genitor_1[0])
#     for i in genitor_1:
#         if i not in vertices_incluidos2:
#             offspring_2.append(i)
#     offspring_2.append(genitor_2[0])
#     return offspring_1, offspring_2
#     pass

#### Mutação

Troca aleatoriamente dois vértices da rota para introduzir diversidade, evitando que o algoritmo fique preso em mínimos locais. É simples, mas eficaz para explorar novas soluções.

In [ ]:
def mutacao_swap(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo in cromossomos:
        # Extrai o caminho do cromossomo (apenas a lista de vértices)
        cromossomo = list(cromossomo[0])

        # Aplica a mutação com probabilidade definida
        if random.random() < probabilidade:
            # Escolhe dois genes aleatórios e troca suas posições
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            cromossomo[gene_1], cromossomo[gene_2] = cromossomo[gene_2], cromossomo[gene_1]
        # Adiciona o cromossomo (mutado ou não) com seu novo custo
        novos_cromossomos.append((cromossomo, calcular_custo(cromossomo, grafo)))

    return novos_cromossomos

Inverte um segmento da rota para criar novas permutações sem alterar demais a solução. Essa mutação é útil para melhorar rotas locais de forma significativa.

In [ ]:
def mutacao_inversao(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo_original_tuple in cromossomos:
        # Extrai a rota
        cromossomo = list(cromossomo_original_tuple[0])

        if random.random() < probabilidade:
            # Escolhe dois genes aleatórios e inverte a ordem do segmento entre eles
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            gene_inicio = min(gene_1, gene_2)
            gene_fim = max(gene_1, gene_2)
            parte_invertida = cromossomo[gene_inicio:gene_fim][::-1]
            novo = cromossomo[:gene_inicio] + parte_invertida + cromossomo[gene_fim:]

            novos_cromossomos.append((novo, calcular_custo(novo,grafo)))
            # print(f"Custo mutado : {calcular_custo(novo, grafo)}")
        else:
            novos_cromossomos.append(cromossomo_original_tuple)
            # Adiciona o cromossomo (mutado ou não) com seu novo custo

    return novos_cromossomos


#### Renovação da população

A função` renovacao_elitismo` vai manter os melhores cromossomos (menor custo) na população, garantindo que soluções de alta qualidade não sejam perdidas.

In [ ]:
def renovacao_elitismo(populacao_atual, offspring, num_populacao):
    # Combina a população atual com os descendentes
    pop = populacao_atual + offspring

    # Ordena todos os cromossomos pelo custo (do menor para o maior)
    pop_ordenada = sorted(pop, key=lambda x: x[1])

    # Mantém apenas os melhores indivíduos para a próxima geração
    return pop_ordenada[:num_populacao]


A funçã `renovacao_torneio` é responsavel pela seleção de forma competitiva: pequenos grupos competem, e o melhor é escolhido. Introduz diversidade, evitando que apenas os melhores de toda a população dominem.

In [ ]:
# Renovacao da populacao por torneio
def renovacao_torneio(populacao_atual, offspring, num_populacao):
    # Usando torneio
    nova_populacao = []
    # Combina população atual com descendentes
    pop = populacao_atual + offspring
    while len(nova_populacao) < num_populacao:
        candidatos = random.choices(pop, k=3)
        # offspring_individuo = random.choice(list(offspring))
        # Escolhe o melhor candidato (menor custo)
        escolhido = min(candidatos, key=lambda x: x[1])
        nova_populacao.append(escolhido)
    return nova_populacao


a função `roleta` seleciona cromossomos com probabilidade proporcional à qualidade da solução: melhores soluções têm maior chance de serem escolhidas, mas soluções piores ainda podem participar, mantendo diversidade.

In [ ]:
def roleta(populacao, offspring, num_populacao):
    populacao = populacao + offspring
    cromossomos = []

    # Calcula o somatório dos custos
    somatorio_custo = sum(map(lambda cromossomo : cromossomo[1], populacao))

    # Normaliza o custo de cada cromossomo para fitness
    values = [solucao[1]/somatorio_custo for solucao in populacao]
    fitness = []
    for val in values:
        min_val = min(values)
        max_val = max(values)
        normalized_data = (max_val - val) / (max_val - min_val)
        fitness.append(normalized_data)
    print(f"Custos : ", list(map(lambda x : x[1], populacao)))
    print(f"Fitness : ", fitness)
    solucoes = list(map(lambda c : c[1], populacao))

    # Seleciona aleatoriamente cromossomos proporcional ao fitness
    return random.choices(populacao, weights=fitness, k=num_populacao)


#### Função principal

Essa função é o fluxo completo do algoritmo genético: ela recebe a população inicial e itera um número definido de vezes, aplicando seleção, cruzamento e mutação, e renovando a população em cada iteração. Ao final, retorna a população otimizada, onde as melhores soluções (rotas com menor custo) tendem a se destacar.

In [ ]:
def algoritmo_genetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao, mutacao, selecao, cruzamento, renovacao):
    populacao_atual = populacao_inicial
    for iter in range(0, numero_iteracoes):
        genitores = selecao(populacao_atual, len(populacao_atual))
        # print(f"Numero de genitores: {len(genitores)}")
        offsprings = cruzamento(genitores, grafo)
        # Ja aplica a taxa de mutacao a populacao
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)
        # print(f"Tamanho do offspring: {len(offsprings)}")
        # print(offsprings)
        populacao_atual = renovacao(populacao_atual, offsprings, len(populacao_atual))
        # print(f"Tamanho da pop: {len(populacao_atual)}")
    return populacao_atual

### Problemas

In [ ]:
def executar_problema(nome_problema, matriz, cromossomos, num_execucoes=20, num_geracoes=100,taxa_mutacao=0.4, mutacao=mutacao_inversao, selecao=elitismo, cruzamento=cruzamento, renovacao=renovacao_elitismo, pasta_resultados="resultados_genetico"):

    # Cria pasta se não existir
    os.makedirs(pasta_resultados, exist_ok=True)

    melhores = []
    tempos = []

    for i in range(num_execucoes):
        inicio = time.perf_counter()
        resultado = algoritmo_genetico(matriz, cromossomos, num_geracoes, taxa_mutacao, mutacao, selecao, cruzamento, renovacao)
        fim = time.perf_counter()
        tempos.append(fim - inicio)
        melhores.append(min(resultado, key=lambda x: x[1]))

    melhores = sorted(melhores, key=lambda x: x[1])


    print(f"Resultados {nome_problema}:")
    print("Melhor solução geral --------")
    print(f"Caminho: {melhores[0][0]}")
    print(f"Custo: {melhores[0][1]}")
    print("---------------------------")
    print("Custo médio: ", sum([custo for _, custo in melhores]) / len(melhores))
    print("Tempo médio: ", sum(tempos) / len(tempos))

    # Salvando em arquivo
    arquivo = os.path.join(pasta_resultados, f"resultado_{nome_problema}.txt")
    with open(arquivo, "w", encoding="utf-8") as f:
        f.write(f"RESULTADOS - {nome_problema}\n")
        f.write(f"CONFIGURAÇÃO:\n")
        f.write(f"  Número de execuções: {num_execucoes}\n")
        f.write(f"  Gerações por execução: {num_geracoes}\n")
        f.write(f"  Taxa de mutação: {taxa_mutacao}\n")
        f.write(f"{'='*60}\nESTATÍSTICAS GERAIS\n{'='*60}\n")
        f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
        f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
        f.write(f"Custo médio: {(sum([custo for _, custo in melhores])/len(melhores)):.2f}\n")
        f.write(f"Tempo médio: {(sum(tempos)/len(tempos)):.2f}s\n")
        f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
        f.write(f"{'='*60}\nTODAS SOLUÇÕES ENCONTRADAS\n{'='*60}\n")
        for idx, (rota, custo) in enumerate(melhores):
            f.write(f"\nTop {idx+1} --------\n")
            f.write(f"Custo: {custo:.2f}\n")
            f.write(f"Rota: {rota}\n")
            f.write("---------------------------\n")

    print(f"Resultados salvos em: {arquivo}\n")


In [ ]:
# Chamando para os 12 problemas
executar_problema("problema_1_km", matriz_problema_km, cromossomos_problema1)
executar_problema("problema_2_min", matriz_problema_min, cromossomos_problema2)
executar_problema("problema_3_km", matriz_problema_km_36, cromossomos_problema3)
executar_problema("problema_4_min", matriz_problema_min_36, cromossomos_problema4)
executar_problema("problema_5_km", matriz_problema_km_24, cromossomos_problema5)
executar_problema("problema_6_min", matriz_problema_min_24, cromossomos_problema6)
executar_problema("problema_7_km", matriz_problema_km_12, cromossomos_problema7)
executar_problema("problema_8_min", matriz_problema_min_12, cromossomos_problema8)
executar_problema("problema_9_km", matriz_problema_km_7, cromossomos_problema9)
executar_problema("problema_10_min", matriz_problema_min_7, cromossomos_problema10)
executar_problema("problema_11_km", matriz_problema_km_6, cromossomos_problema11)
executar_problema("problema_12_min", matriz_problema_min_6, cromossomos_problema12)



Resultados problema_1_km:
Melhor solução geral --------
Caminho: [1, 10, 12, 9, 8, 21, 22, 48, 40, 36, 25, 31, 17, 7, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 11, 2, 23, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Custo: 2046.1999999999998
---------------------------
Custo médio:  2046.2
Tempo médio:  1.6609410305999517
Resultados salvos em: resultados_genetico/resultado_problema_1_km.txt

Resultados problema_2_min:
Melhor solução geral --------
Caminho: [1, 10, 11, 5, 2, 3, 4, 30, 6, 27, 41, 44, 28, 15, 18, 45, 46, 14, 43, 29, 24, 13, 37, 47, 39, 35, 42, 26, 32, 16, 34, 33, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 12, 9, 8, 21, 23, 1]
Custo: 2004.0
---------------------------
Custo médio:  2071.0
Tempo médio:  1.7330695135001406
Resultados salvos em: resultados_genetico/resultado_problema_2_min.txt

Resultados problema_3_km:
Melhor solução geral --------
Caminho: [1, 10, 12, 9, 8, 21, 22, 36, 25, 31, 17, 7, 19, 20, 33, 16, 34, 32, 26, 35, 13, 24

## Algoritmo Memetico

Um algoritmo memético é uma evolução do algoritmo genético que combina evolução populacional (seleção, cruzamento e mutação) com busca local para refinar soluções individuais. Ele explora globalmente o espaço de soluções e, ao mesmo tempo, melhora cada indivíduo com heurísticas, geralmente gerando resultados de maior qualidade e mais rápido que um algoritmo genético puro.

O código implementa um algoritmo memético que, a cada iteração, seleciona genitores da população atual, gera novos indivíduos via cruzamento e mutação, aplica busca local para melhorar algumas soluções e, finalmente, renova a população. Ele acompanha o melhor indivíduo de cada geração e registra o tempo de execução por iteração.



In [ ]:
def algoritmo_memetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao, mutacao, selecao, cruzamento, renovacao, heuristicas_busca_local, probabilidade_busca_local=0.2):
    populacao_atual = populacao_inicial
    melhores = []
    tempos = []

    # Calcula o número de genitores para a seleção (metade do tamanho da população)
    n_genitores = len(populacao_atual) // 2
    print(f"Numero de genitores: {n_genitores}")

    # Loop principal do algoritmo memético, iterando pelo número de gerações
    for iteracao in range(numero_iteracoes):

        # Registra o tempo de início da iteração
        tempo_começo = time.perf_counter()

        # Seleciona os genitores a partir da população atual usando a função de seleção especificada
        genitores = selecao(populacao_atual, n_genitores)

        # Realiza o cruzamento entre os genitores para gerar descendentes (offsprings)
        offsprings = cruzamento(genitores, grafo)

        # Aplica mutação aos descendentes com a taxa de mutação especificada
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)
        novos_offsprings = []

        for item in offsprings:
            # Extrai o indivíduo da tupla (solução, custo) ou diretamente se não for uma tupla
            if isinstance(item, tuple):
                individuo = item[0]
            else:
                individuo = item

            # Calcula o custo do indivíduo (pode ser o custo original ou o custo mutado)
            custo_individuo = calcular_custo(individuo, grafo)

            # Aplica a busca local com uma determinada probabilidade
            if random.random() < probabilidade_busca_local:
                melhorou = False
                # Tenta melhorar o indivíduo com cada heurística de busca local
                for heuristica in heuristicas_busca_local:
                    nova_rota, novo_custo = busca_local_primeira_melhoria(individuo, custo_individuo, heuristica, grafo)
                    if novo_custo < custo_individuo:
                        individuo = nova_rota
                        custo_individuo = novo_custo
                        melhorou = True

            # Adiciona o indivíduo (potencialmente melhorado pela busca local) à lista de novos descendentes
            novos_offsprings.append((individuo, custo_individuo))

        # Registra o tempo de fim da iteração
        tempo_fim = time.perf_counter()
        tempos.append(tempo_fim - tempo_começo)

        # print(f"Tamanho do offspring: {len(novos_offsprings)}")
        # print(novos_offsprings)
        print(f"Tamanho da pop: {len(populacao_atual)}")

        # Atualiza a população usando a função de renovação especificada (elitismo, torneio, etc.)
        populacao_atual = renovacao(populacao_atual, novos_offsprings, len(populacao_atual))

        # Armazena o melhor custo da geração atual (o primeiro da população ordenada)
        melhor_custo_geracao = populacao_atual[0][1]
        melhores.append(populacao_atual[0])

        print(f"Geração {iteracao + 1}/{numero_iteracoes} | Melhor Custo: {melhor_custo_geracao:.2f} | Pop: {len(populacao_atual)}\n")

    return populacao_atual, melhores, tempos

### Problemas

In [ ]:
problemas = [
    {'nome': 'problema_1_km', 'matriz': matriz_problema_km, 'cromossomos': cromossomos_problema1},
    {'nome': 'problema_2_min', 'matriz': matriz_problema_min, 'cromossomos': cromossomos_problema2},
    {'nome': 'problema_3_km', 'matriz': matriz_problema_km_36, 'cromossomos': cromossomos_problema3},
    {'nome': 'problema_4_min', 'matriz': matriz_problema_min_36, 'cromossomos': cromossomos_problema4},
    {'nome': 'problema_5_km', 'matriz': matriz_problema_km_24, 'cromossomos': cromossomos_problema5},
    {'nome': 'problema_6_min', 'matriz': matriz_problema_min_24, 'cromossomos': cromossomos_problema6},
    {'nome': 'problema_7_km', 'matriz': matriz_problema_km_12, 'cromossomos': cromossomos_problema7},
    {'nome': 'problema_8_min', 'matriz': matriz_problema_min_12, 'cromossomos': cromossomos_problema8},
    {'nome': 'problema_9_km', 'matriz': matriz_problema_km_7, 'cromossomos': cromossomos_problema9},
    {'nome': 'problema_10_min', 'matriz': matriz_problema_min_7, 'cromossomos': cromossomos_problema10},
    {'nome': 'problema_11_km', 'matriz': matriz_problema_km_6, 'cromossomos': cromossomos_problema11},
    {'nome': 'problema_12_min', 'matriz': matriz_problema_min_6, 'cromossomos': cromossomos_problema12}
]

print(f"Foram criadas {len(problemas)} configurações de problema.")

pasta_resultados_memetico = "resultados_memetico"
os.makedirs(pasta_resultados_memetico, exist_ok=True)

for config in problemas:
    nome_problema = config['nome']
    matriz = config['matriz']
    cromossomos = config['cromossomos']

    print(f"\nExecutando Algoritmo Memético para {nome_problema}...")

    # Executa o algoritmo memético
    populacao_final, melhores, tempos = algoritmo_memetico(grafo=matriz, populacao_inicial=cromossomos, numero_iteracoes=20, taxa_mutacao=0.3, mutacao=mutacao_inversao, selecao=elitismo, cruzamento=cruzamento,renovacao=renovacao_elitismo,heuristicas_busca_local=[swap, shift, inversao])

    # Salva os resultados em arquivo
    arquivo_saida = os.path.join(pasta_resultados_memetico, f"resultado_{nome_problema}.txt")
    with open(arquivo_saida, 'w', encoding='utf-8') as f:
        f.write(f"RESULTADOS - {nome_problema.upper()}\n")
        f.write(f"\nCONFIGURAÇÃO:\n")
        f.write(f"  Número de gerações: 20\n")
        f.write(f"  Taxa de mutação: 0.3\n")
        f.write(f"{'='*60}\nESTATÍSTICAS GERAIS\n{'='*60}\n")

        if melhores:
            melhor = min(melhores, key=lambda x: x[1])
            f.write(f"Melhor caminho encontrado: {melhor[0]}\n")
            f.write(f"Melhor custo encontrado: {melhor[1]:.2f}\n")
            f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
        else:
            f.write("Nenhuma solução encontrada.\n")
            f.write(f"Custo médio: N/A\n")

        f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
        f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")

        f.write(f"{'='*60}\nTODAS AS MELHORES SOLUÇÕES POR GERAÇÃO\n{'='*60}\n")
        if melhores:
            for idx, (rota, custo) in enumerate(melhores):
                f.write(f"\nGeração {idx+1} --------\n")
                f.write(f"Custo: {custo:.2f}\n")
                f.write(f"Rota: {rota}\n")
                f.write("---------------------------\n")
        else:
            f.write("Nenhuma solução por geração encontrada.\n")

    print(f"Resultados salvos em: {arquivo_saida}")

Foram criadas 12 configurações de problema.

Executando Algoritmo Memético para problema_1_km...
Numero de genitores: 20
Tamanho da pop: 40
Geração 1/20 | Melhor Custo: 1960.10 | Pop: 40

Tamanho da pop: 40
Geração 2/20 | Melhor Custo: 1960.10 | Pop: 40

Tamanho da pop: 40
Geração 3/20 | Melhor Custo: 1960.10 | Pop: 40

Tamanho da pop: 40
Geração 4/20 | Melhor Custo: 1950.60 | Pop: 40

Tamanho da pop: 40
Geração 5/20 | Melhor Custo: 1950.60 | Pop: 40

Tamanho da pop: 40
Geração 6/20 | Melhor Custo: 1950.60 | Pop: 40

Tamanho da pop: 40
Geração 7/20 | Melhor Custo: 1950.60 | Pop: 40

Tamanho da pop: 40
Geração 8/20 | Melhor Custo: 1950.60 | Pop: 40

Tamanho da pop: 40
Geração 9/20 | Melhor Custo: 1950.60 | Pop: 40

Tamanho da pop: 40
Geração 10/20 | Melhor Custo: 1950.60 | Pop: 40

Tamanho da pop: 40
Geração 11/20 | Melhor Custo: 1950.60 | Pop: 40

Tamanho da pop: 40
Geração 12/20 | Melhor Custo: 1950.60 | Pop: 40

Tamanho da pop: 40
Geração 13/20 | Melhor Custo: 1950.60 | Pop: 40

Tama